In [1]:
import re
import os
import itertools
import numpy as np
import pandas as pd
from typing import Any, Dict
import random
import yaml
import argparse
import glob
from tqdm import tqdm

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores.utils import DistanceStrategy
import faiss
import pdfplumber
import torch
from sentence_transformers import CrossEncoder
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import warnings

2025-08-20 15:55:53.261079: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-20 15:55:53.268461: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755672953.277448  606851 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755672953.280139  606851 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-20 15:55:53.289557: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
base_path = "./law"
pdf_path = [base_path + f"/{i}" for i in os.listdir(base_path)]

pdf_text = {
    i : "" for i in os.listdir(base_path)
}

for i, key in zip(pdf_path, pdf_text.keys()):
    text = ""
    with pdfplumber.open(i) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n" 
    
    pdf_text[key] = text

Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss 

In [3]:
# for key, i in pdf_text.items():
#     print(i)

print(pdf_text.keys())

dict_keys(['전자금융거래법.pdf', '금융회사의 정보처리 업무 위탁에 관한 규정.pdf', '금융위원회의 설치 등에 관한 법률.pdf', '전자상거래 등에서의 소비자보호에 관한 법률.pdf', '정보통신망 이용촉진 및 정보보호 등에 관한 법률.pdf', '개인정보 보호법.pdf', '전자문서 및 전자거래 기본법.pdf', '소득세법.pdf', '여신전문금융업법.pdf', '전자서명법.pdf', '신용정보의 이용 및 보호에 관한 법률.pdf', '전자정부법.pdf', '정보보호산업의 진흥에 관한 법률 시행규칙.pdf', '전자금융감독규정.pdf', '정보통신기반 보호법.pdf', '사이버안보 업무규정.pdf', '산업표준화법.pdf', '은행법.pdf', '전자금융거래법 시행령.pdf', '금융기관검사및제재에관한규정.pdf', '정보보호산업의 진흥에 관한 법률.pdf'])


In [4]:
def split_by_article(text):
    # '제 N조' 앞에서 split
    pattern = r'(제\s*\d+조(?:의\d+)?\s*\([^)]+\)[\s\S]*?)(?=제\s*\d+조(?:의\d+)?\s*\([^)]+\)|\Z)'
    return re.findall(pattern, text)

pdf_split_data = []

for article, text in pdf_text.items():
    split_text = split_by_article(text)
    for i in split_text:
        pdf_split_data.append(f"{article[:-4]} {i}")

pdf_split_data[:5]

['전자금융거래법 제1조(목적) 이 법은 전자금융거래의 법률관계를 명확히 하여 전자금융거래의 안전성과 신뢰성을 확보함과 아울러 전\n자금융업의 건전한 발전을 위한 기반조성을 함으로써 국민의 금융편의를 꾀하고 국민경제의 발전에 이바지함을 목\n적으로 한다.\n',
 '전자금융거래법 제2조(정의) 이 법에서 사용하는 용어의 정의는 다음과 같다. <개정 2007. 4. 27., 2008. 2. 29., 2012. 3. 21., 2012. 6. 1.,\n2013. 5. 22., 2020. 6. 9., 2023. 9. 14.>\n1. “전자금융거래”라 함은 금융회사 또는 전자금융업자가 전자적 장치를 통하여 금융상품 및 서비스를 제공(이하\n“전자금융업무”라 한다)하고, 이용자가 금융회사 또는 전자금융업자의 종사자와 직접 대면하거나 의사소통을 하\n지 아니하고 자동화된 방식으로 이를 이용하는 거래를 말한다.\n2. “전자지급거래”라 함은 자금을 주는 자(이하 “지급인”이라 한다)가 금융회사 또는 전자금융업자로 하여금 전자지\n급수단을 이용하여 자금을 받는 자(이하 “수취인”이라 한다)에게 자금을 이동하게 하는 전자금융거래를 말한다.\n3. “금융회사”란 다음 각 목의 어느 하나에 해당하는 기관이나 단체 또는 사업자를 말한다.\n가. 「금융위원회의 설치 등에 관한 법률」 제38조제1호부터 제5호까지, 제7호 및 제8호에 해당하는 기관\n나. 「여신전문금융업법」에 따른 여신전문금융회사\n다. 「우체국예금ㆍ보험에 관한 법률」에 따른 체신관서\n라. 「새마을금고법」에 따른 새마을금고 및 새마을금고중앙회\n마. 그 밖에 법률의 규정에 따라 금융업 및 금융 관련 업무를 행하는 기관이나 단체 또는 사업자로서 대통령령이\n정하는 자\n4. “전자금융업자”라 함은 제28조의 규정에 따라 허가를 받거나 등록을 한 자(금융회사는 제외한다)를 말한다.\n5. “전자금융보조업자”라 함은 금융회사 또는 전자금융업자를 위하여 전자금융거래를 보조하거나 그 일부를 대행하\n는 업무를 행하는 자 또는 

In [5]:
tech_data = pd.read_csv("/home/aicontest/finance/tech/tech_data_noise.csv")

tech_array = list(tech_data['data'])
clean_array = []
for i in tech_array:
    if len(i) > 1000:
        clean_array.append(". ".join(list(i[:800].split("."))[:-1]))
    else:
        clean_array.append(i)
        
pdf_split_data.extend(clean_array)

In [6]:
max_len = float("-inf")
min_len = float("inf")
total_len = 0

for i in pdf_split_data:
    length = len(i)

    if length > max_len:
        max_len = length

    if length < min_len:
        min_len = length

    total_len += length

print(f"total len: {total_len}")
print(f"max length : {max_len}")
print(f"min length : {min_len}")
print(f"mean length : {total_len/len(pdf_split_data)}")

total len: 25833653
max length : 7211
min length : 0
mean length : 472.4515910753475


In [7]:
pdf_split_data_recursive = []

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

for i in pdf_split_data:
    clean_text = re.sub(r'\s+', ' ', i).strip()
    chunks = splitter.split_text(clean_text)
    pdf_split_data_recursive.extend(chunks)

pdf_split_data_recursive = [re.sub(r"[<\[].*?[>\]]", "", text).strip() for text in pdf_split_data_recursive]

pdf_split_data_recursive[:10]

['전자금융거래법 제1조(목적) 이 법은 전자금융거래의 법률관계를 명확히 하여 전자금융거래의 안전성과 신뢰성을 확보함과 아울러 전 자금융업의 건전한 발전을 위한 기반조성을 함으로써 국민의 금융편의를 꾀하고 국민경제의 발전에 이바지함을 목 적으로 한다.',
 '전자금융거래법 제2조(정의) 이 법에서 사용하는 용어의 정의는 다음과 같다.  1. “전자금융거래”라 함은 금융회사 또는 전자금융업자가 전자적 장치를 통하여 금융상품 및 서비스를 제공(이하 “전자금융업무”라 한다)하고, 이용자가 금융회사 또는 전자금융업자의 종사자와 직접 대면하거나 의사소통을 하 지 아니하고 자동화된 방식으로 이를 이용하는 거래를 말한다. 2. “전자지급거래”라 함은 자금을 주는 자(이하 “지급인”이라 한다)가 금융회사 또는 전자금융업자로 하여금 전자지 급수단을 이용하여 자금을 받는 자(이하 “수취인”이라 한다)에게 자금을 이동하게 하는 전자금융거래를 말한다. 3. “금융회사”란 다음 각 목의 어느 하나에 해당하는 기관이나 단체 또는 사업자를 말한다. 가. 「금융위원회의 설치 등에 관한 법률」',
 '“전자지급거래”라 함은 자금을 주는 자(이하 “지급인”이라 한다)가 금융회사 또는 전자금융업자로 하여금 전자지 급수단을 이용하여 자금을 받는 자(이하 “수취인”이라 한다)에게 자금을 이동하게 하는 전자금융거래를 말한다. 3. “금융회사”란 다음 각 목의 어느 하나에 해당하는 기관이나 단체 또는 사업자를 말한다. 가. 「금융위원회의 설치 등에 관한 법률」 제38조제1호부터 제5호까지, 제7호 및 제8호에 해당하는 기관 나. 「여신전문금융업법」에 따른 여신전문금융회사 다. 「우체국예금ㆍ보험에 관한 법률」에 따른 체신관서 라. 「새마을금고법」에 따른 새마을금고 및 새마을금고중앙회 마. 그 밖에 법률의 규정에 따라 금융업 및 금융 관련 업무를 행하는 기관이나 단체 또는 사업자로서 대통령령이 정하는 자 4. “전자금융업자”라 함은 제28조의 규정에 따라 허가를 받거나 등록을 한 자(금융회사는 제외한다)

In [8]:
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

vector_store = FAISS.from_texts(pdf_split_data_recursive, embedding, distance_strategy = DistanceStrategy.COSINE)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":10})

/tmp/ipykernel_606851/2243535589.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)
/home/aicontest/anaconda3/envs/construct/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [9]:
query = """정보통신망법 제47조의4에 따라 주요정보통신서비스 제공자가 정보통신망에 중대한 침해사고가 발생할 경우 취할 수 있는 조치는?
1 이용자에게 보호조치를 요청한다.
2 이용자의 정보를 삭제한다.
3 이용자의 정보를 공개한다.
4 이용자의 계정을 영구적으로 정지한다."""

text = retriever.get_relevant_documents(query)

for i in text:
    print(i.page_content)

개인정보 보호법 제34조의2(노출된 개인정보의 삭제ㆍ차단) ① 개인정보처리자는 고유식별정보, 계좌정보, 신용카드정보 등 개인정보가 정보통신망을 통하여 공중(公衆)에 노출되지 아니하도록 하여야 한다.  ② 개인정보처리자는 공중에 노출된 개인정보에 대하여 보호위원회 또는 대통령령으로 지정한 전문기관의 요청이 있는 경우에는 해당 정보를 삭제하거나 차단하는 등 필요한 조치를 하여야 한다.  ] 제5장 정보주체의 권리 보장
신용정보의 이용 및 보호에 관한 법률 제45조의4(보호위원회의 시정조치) 보호위원회는 상거래정보보호규정과 관련하여 개인신용정보가 침해되었다고 판단 할 상당한 근거가 있고 이를 방치할 경우 회복하기 어려운 피해가 발생할 우려가 있다고 인정되면 상거래기업 및 법 인에 대하여 다음 각 호에 해당하는 조치를 명할 수 있다. 1. 개인신용정보 침해행위의 중지 2. 개인신용정보 처리의 일시적인 정지 3. 그 밖에 개인정보의 보호 및 침해 방지를 위하여 필요한 조치
있다고 판단되 는 경우 2. 외부에서 발생한 침해사고로 집적된 정보통신시설에 심각한 장애가 발생할 우려가 있다고 판단되는 경우 3. 중대한 침해사고가 발생하여 과학기술정보통신부장관이나 한국인터넷진흥원이 요청하는 경우 ② 집적정보통신시설 사업자는 제1항에 따라 해당 서비스의 제공을 중단하는 경우에는 중단사유, 발생일시, 기간 및 내용 등을 구체적으로 밝혀 시설이용자에게 즉시 알려야 한다. ③ 집적정보통신시설 사업자는 중단사유가 없어지면 즉시 해당 서비스의 제공을 재개하여야 한다.  제46조의3 삭제
법률 ③ 과학기술정보통신부장관 또는 한국인터넷진흥원은 제2항에 따른 신고를 받거나 제1항을 위반한 사실을 알게 되 면 다음 각 호의 필요한 조치를 하여야 한다. 1. 위반 사실에 관한 정보의 수집ㆍ전파 2. 유사 피해에 대한 예보ㆍ경보 3. 정보통신서비스 제공자에게 다음 각 목의 사항 중 전부 또는 일부를 요청하는 등 피해 예방 및 피해 확산을 방지 하기 위한 긴급조치 가. 접속경로의 차단 나. 제1항의 위

/tmp/ipykernel_606851/3849381724.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  text = retriever.get_relevant_documents(query)
/home/aicontest/anaconda3/envs/construct/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [10]:
# 예시: 쿼리 + 문서 리스트
docs = [i.page_content for i in text]

# Query-document 쌍 만들기
pairs = [(query, doc) for doc in docs]

def exp_normalize(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()

model_path = "Dongjin-kr/ko-reranker"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    scores = exp_normalize(scores.numpy())
    
    for k, v in zip(pairs, scores):
        print(f"{k} : {v}")


/home/aicontest/anaconda3/envs/construct/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


('정보통신망법 제47조의4에 따라 주요정보통신서비스 제공자가 정보통신망에 중대한 침해사고가 발생할 경우 취할 수 있는 조치는?\n1 이용자에게 보호조치를 요청한다.\n2 이용자의 정보를 삭제한다.\n3 이용자의 정보를 공개한다.\n4 이용자의 계정을 영구적으로 정지한다.', '개인정보 보호법 제34조의2(노출된 개인정보의 삭제ㆍ차단) ① 개인정보처리자는 고유식별정보, 계좌정보, 신용카드정보 등 개인정보가 정보통신망을 통하여 공중(公衆)에 노출되지 아니하도록 하여야 한다.  ② 개인정보처리자는 공중에 노출된 개인정보에 대하여 보호위원회 또는 대통령령으로 지정한 전문기관의 요청이 있는 경우에는 해당 정보를 삭제하거나 차단하는 등 필요한 조치를 하여야 한다.  ] 제5장 정보주체의 권리 보장') : 2.2485279259853996e-05
('정보통신망법 제47조의4에 따라 주요정보통신서비스 제공자가 정보통신망에 중대한 침해사고가 발생할 경우 취할 수 있는 조치는?\n1 이용자에게 보호조치를 요청한다.\n2 이용자의 정보를 삭제한다.\n3 이용자의 정보를 공개한다.\n4 이용자의 계정을 영구적으로 정지한다.', '신용정보의 이용 및 보호에 관한 법률 제45조의4(보호위원회의 시정조치) 보호위원회는 상거래정보보호규정과 관련하여 개인신용정보가 침해되었다고 판단 할 상당한 근거가 있고 이를 방치할 경우 회복하기 어려운 피해가 발생할 우려가 있다고 인정되면 상거래기업 및 법 인에 대하여 다음 각 호에 해당하는 조치를 명할 수 있다. 1. 개인신용정보 침해행위의 중지 2. 개인신용정보 처리의 일시적인 정지 3. 그 밖에 개인정보의 보호 및 침해 방지를 위하여 필요한 조치') : 0.00043973454739898443
('정보통신망법 제47조의4에 따라 주요정보통신서비스 제공자가 정보통신망에 중대한 침해사고가 발생할 경우 취할 수 있는 조치는?\n1 이용자에게 보호조치를 요청한다.\n2 이용자의 정보를 삭제한다.\n3 이용자의 정보를 공개한다.\n4 이용자의 계정을 영구적

In [11]:
warnings.filterwarnings("ignore", category=FutureWarning)
test_df = pd.read_csv("./data/test.csv")

submission_dict = {
    "ID" : [],
    "Question" : [],
    "RagInfo" : [],
    "score": []
}

for _, i in tqdm(test_df.iterrows(), total=len(test_df), desc="Rag inference..."):
    question = i["Question"]
    text = retriever.get_relevant_documents(question)

    docs = [i.page_content for i in text]
    pairs = [(question, doc) for doc in docs]

    with torch.no_grad():
        inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
        scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
        scores = exp_normalize(scores.numpy())
    
    Rag_information = pairs[np.argmax(scores)]

    submission_dict["ID"].append(i["ID"])
    submission_dict["Question"].append(question)
    submission_dict["RagInfo"].append(Rag_information[1])
    submission_dict["score"].append(max(scores))

submission_df = pd.DataFrame(submission_dict)
submission_df.to_csv("./data/raginfo_tech.csv")

Rag inference...: 100%|██████████| 515/515 [27:38<00:00,  3.22s/it]


In [ ]:
test = pd.read_csv("/home/aicontest/finance/data/raginfo_tech.csv")
test_list = [float(i) for i in test['score']]
print(sum(test_list)/len(test_list))

0.7057003752135917


: 